In [ ]:
import os
import pandas
import threading
import time

In [ ]:
import snowflake.connector

In [ ]:
os.environ['ACCOUNT'] = "ek38061"
os.environ['REGION'] = "central-india.azure"
os.environ['USER'] = "sohankotkar3"
os.environ['PASSWORD'] = "Sohan321"
os.environ['ROLE'] = "ACCOUNTADMIN"
os.environ['WAREHOUSE'] = "WH_XSMALL"
os.environ['DATABASE'] = "FIVEKASSETS"
os.environ['SCHEMA'] = "PUBLIC"

In [ ]:
connection = snowflake.connector.connect(
    user = os.environ['USER'],
    password = os.environ['PASSWORD'],
    account = os.environ['ACCOUNT'],
    region = os.environ['REGION'],
    database = os.environ['DATABASE'],
    warehouse = os.environ['WAREHOUSE'],
    schema = os.environ['SCHEMA'],
    role = os.environ['ROLE']
)

In [ ]:
num_clusters = 1 #Now Warehouse has only 1 cluster and it won't add any extra cluster when load increases.
concurrent_queries = 441528

In [ ]:
# set the maximum number of clusters available to the warehouse
def set_max_clusters(i):
    connection.cursor().execute(
        'alter warehouse "' + os.environ['WAREHOUSE'] +
        '" set min_cluster_count = 1 max_cluster_count = ' + str(i)
    )
    connection.cursor().execute(
        'ALTER SESSION SET USE_CACHED_RESULT = FALSE'
    )

In [ ]:
queries = [
    'select id, time, isValid, dataStatus, nodeStatus, value from Fivekassets.PUBLIC.staticdata_big_tw3 where id = (SELECT id from Fivekasset.PUBLIC.distid where seq = %s )',
    'select id, time, isValid, dataStatus, nodeStatus, value from Fivekassets.PUBLIC.staticdata_big_tw3 where id = (SELECT id from Fivekasset.PUBLIC.distid where seq = %s ) and time =\'2018-05-10 13:45:46.000\' ',
    'select distinct fullData.* from Fivekassets.PUBLIC.staticdata_big_tw3 fullData inner join ( select id, min(time) as minMaxTime from ( select id, time from Fivekassets.PUBLIC.staticdata_big_tw3 where id = (SELECT id from Fivekasset.PUBLIC.distid where seq = %s ) and time >= \'2018-02-19 13:34:55.000\' and time <= \'2021-11-03 11:05:10.000\' ) as minimumData group by id ) as minMaxData on ( fullData.id = minMaxData.id and fullData.time = minMaxData.minMaxTime )',
    'select distinct fullData.* from Fivekassets.PUBLIC.staticdata_big_tw3 fullData inner join ( select id, max(time) as minMaxTime from ( select id, time from Fivekassets.PUBLIC.staticdata_big_tw3 where id = (SELECT id from Fivekasset.PUBLIC.distid where seq = %s ) and time >= \'2018-02-19 13:34:55.000\' and time <= \'2021-11-03 11:05:10.000\' ) as minimumData group by id ) as minMaxData on ( fullData.id = minMaxData.id and fullData.time = minMaxData.minMaxTime )'
]

In [ ]:
individual_results = []
query_set_results = []

class ConcurrentQuery (threading.Thread):
    
    def __init__(self, total, counter, clusters, seq):
        threading.Thread.__init__(self)
        self.counter = counter
        self.total = total
        self.clusters = clusters
        self.seq = seq

    def run(self):
        cur = connection.cursor()
        query_number = 0
        print("seq = "+str(self.seq))
        qs_start_ts = time.time()
#        for seq in range(11,21):
        for query in queries:
            start_ts = time.time()
            cur.execute(query, str(self.seq))
            end_ts = time.time()
            individual_results.append(
                {
                    'clusters': self.clusters,
                    'total': self.total,
                    'counter': self.counter,
                    'query_number': query_number,
                    'start_time': time.strftime('%Y-%m-%dT%H:%M:%S', time.localtime(start_ts)),
                    'execution_time': end_ts - start_ts,
                    'query_id': cur.sfqid
                }
            )
            query_number = query_number + 1
            self.seq = self.seq + 1
            time.sleep(seq%5) # Here we are mimicing real world scenario where we expect some gap between 2 consecutive queries.
        qs_end_ts = time.time()
        query_set_results.append(
            {
                'clusters': self.clusters,
                'total': self.total,
                'counter': self.counter,
                'start_time': time.strftime('%Y-%m-%dT%H:%M:%S', time.localtime(qs_start_ts)),
                'execution_time': qs_end_ts - qs_start_ts,
                'query_id': cur.sfqid
            }
        )

In [ ]:
set_max_clusters(num_clusters)

time.sleep(10) # it takes a bit of time for a cluster to spin up

print(time.strftime('%Y-%m-%d %H:%M:%S'))

threads = []
incre = len(queries)
seq = 1

for query_num in range(0, concurrent_queries):
    thread = ConcurrentQuery(concurrent_queries, query_num, num_clusters, seq)
    threads.append(thread)
    seq = seq + incre
print("Final seq = "+str(seq))
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(time.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
df = pandas.DataFrame(individual_results)
df.to_csv('concurrency_test_result3_individual_2.csv', index=False)

In [ ]:
df1 = pandas.DataFrame(query_set_results)
df1.to_csv('concurrency_test_result3_queryset_2.csv', index=False)

In [ ]:
print(time.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
cur = connection.cursor()

In [ ]:
cur.execute("select id, time, isValid, dataStatus, nodeStatus, value from Fivekassets.PUBLIC.staticdata_azure where id = (SELECT id from Fivekasset.PUBLIC.distid where seq = %s )", str(19))

In [ ]:
len(queries)

In [ ]:
q = 34

In [ ]:
time.sleep(q%5)